# Creating green and blue space indicators from Sentinel-2 data

The notebook will estimate the following statistics for each household, as defined as the Unique Property Reference Number (UPRN), across Cheshire and Merseyside:
* Normalised Difference Vegetation Index (NDVI)
* Enhanced Vegetation Index (EVI)
* Normalised Water Difference Idex (NWDI)


## Set up the environment

We will use Google Earth Engine (GEE) to access Sentinel-2 images. The strength of using GEE is that we can store and process all the images in the cloud, saving space. Let's start with installing GEE and arrow.

In [ ]:
# Install required packages
# ! pip install earthengine-api # Comes already installed in Colab so leave here for local running
! pip install pyarrow

Next we need to set up Python for the neccessary packages and link the notebook to our set up Groundswell GEE project.

In [1]:
# Load required libaries
import ee
import geopandas as gpd
import pandas as pd
import time
from shapely.geometry import mapping

# Set up Google Earth Engine (GEE) module
ee.Authenticate(auth_mode = "colab") # Links GEE to your Google Account and defines that working in Colab (can change to 'localhost' if working on a local machine)
ee.Initialize(project = "ee-groundswelluk") # Link to the registered project within GEE

## Process images
The first key processing task that we need to undertake is to access the satellite imagery and process it into the neccessary format for generating our indicators. As such, we need to: (i) select which satellite we want to use (Sentinel-2), (ii) define the region and extent of images we need (Cheshire and Merseyside), (iii) select the time period (2024) of interest, (iv) remove any clouds from images so that they do not affect any values, (v) extract the information that we want to extract from the images (NDVI, EVI and NWDI), and (vi) take the median value for the whole time period (i.e., combine the information across multiple images).

From my testing, it is more computationally efficient to divide our processing by Local Authority rather than trying to do it for the whole region in one go. So below, we will create and process the image for each Local Authority seperately and then save these to Google Earth Engine. We will link the image values to the household information later.

As a rough time guide, it takes ~7 minutes to process all of the images for Liverpool in 2023 in Google Colab / Earth Engine, so should take 35-40 minutes in total.

In [ ]:
# Define bounding box geometries for each area
liverpool_bbox = ee.Geometry.Rectangle([-3.012314, 53.324927, -2.808037, 53.479261])
sefton_bbox = ee.Geometry.Rectangle([-3.123550, 53.433674, -2.881851, 53.686949])
knowsley_bbox = ee.Geometry.Rectangle([-2.940216, 53.313237, -2.682724, 53.512143])
wirral_bbox = ee.Geometry.Rectangle([-3.204575, 53.304826, -2.918587, 53.444104])
halton_bbox = ee.Geometry.Rectangle([-2.827263, 53.301954, -2.594833, 53.402982])
warrington_bbox = ee.Geometry.Rectangle([-2.698860, 53.316518, -2.455444, 53.482325])
chester_cheshire_west_bbox = ee.Geometry.Rectangle([-3.124237, 52.964770, -2.327042, 53.326772])
cheshire_east_bbox = ee.Geometry.Rectangle([-2.736969, 52.921323, -1.976166, 53.423446])
st_helens_bbox = ee.Geometry.Rectangle([-2.845459, 53.351781, -2.526855, 53.555810])

# List of bounding boxes
areas = [
    ('Liverpool', liverpool_bbox),
    ('Sefton', sefton_bbox),
    ('Knowsley', knowsley_bbox),
    ('Wirral', wirral_bbox),
    ('Halton', halton_bbox),
    ('Warrington', warrington_bbox),
    ('Chester_and_Cheshire_West', chester_cheshire_west_bbox),
    ('Cheshire_East', cheshire_east_bbox),
    ('St_Helens', st_helens_bbox)
]

# Cloud masking function for Sentinel-2
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load Sentinel-2 surface reflectance data
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2024-01-01', '2024-07-02') \
    .map(maskS2clouds)

# Calculate NDVI, EVI, and NDWI
def addIndices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands([ndvi, evi, ndwi])

# Iterate over each area, calculate median value and export the data to Earth Engine assets
for area_name, bbox in areas:
    print(f'Processing {area_name}...')

    # Filter collection by bounding box
    area_collection = collection.filterBounds(bbox)

    # Calculate indices
    dataset = area_collection.map(addIndices)

    # Calculate median value for each index for the time period within the bounding box
    median = dataset.median().clip(bbox)

    # Select the bands of interest
    bands = median.select(['NDVI', 'EVI', 'NDWI'])

    # Export the data to an Earth Engine asset
    task = ee.batch.Export.image.toAsset(
        image=bands,
        description=f'{area_name}_Vegetation_Water_Indices',
        assetId=f'projects/ee-groundswelluk/assets/2024/{area_name}_Vegetation_Water_Indices', # Where to save on earth engine
        region=bbox,
        scale=10,
        maxPixels=1e13
    )
    task.start()

    # Monitor the task (let know progress or print any errors)
    def monitor_task(task):
        while task.active():
            print(f'Task {task.status()["description"]} is {task.status()["state"]}') # Print current status of the task (ready or running)
            time.sleep(60) # Wait 60 seconds before re-running above
        status = task.status()
        print(f'Task {status["description"]} completed with status: {status["state"]}') # Print if completed task
        if status['state'] != 'COMPLETED': # If not completed
            print('Error:', status['error_message'])  # Then print the error associated with this

    monitor_task(task)



## Create household-level indicators
Here we take the processed images that we have for each area, load in the points for households, and then estimate the average score for a 300m buffer around each point.

I have previously processed the households (UPRNs) so that they are in the neccessary format to help make this next step most efficient here - using `get_uprns_cm.R` (make sure you have run this first). This includes: (i) Using only TOIDs (Topographic Identifers) rather than UPRNs (UPRNs are nested within TOIDs which represent the unique building rather than household - e.g., UPRNs would be the individual flats and the TOID would be the single building). This helps to avoid duplication of estimation for multiple UPRNs in the same location (using TOIDs leads to 23% fewer points to process). (ii)  Calculating a 300m buffer around each point seperately, since this is a computationally expensive task for 1.2M points to do here within this script.

We can now estimate the median values of NDVI, EDI and NWDI for all inputs. The code below splits this task up by Local Authority and processes each individually. This was done here because it helps to minimise the memory needs of these operations which can lead to Colab crashing. The code below takes 46 minutes to run in total (about ~7 minutes per Local Authority, depending on their size).

In [ ]:
# Define the mapping of Local Authority codes to shapefile paths
shapefile_paths = {
    'E06000049': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E06000049.shp',
    'E06000050': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E06000050.shp',
    'E06000006': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E06000006.shp',
    'E08000011': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E08000011.shp',
    'E08000012': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E08000012.shp',
    'E08000014': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E08000014.shp',
    'E08000013': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E08000013.shp',
    'E06000007': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E06000007.shp',
    'E08000015': '/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/toid_buffers_by_lad/toid_buffer_E08000015.shp'
}

def calculate_medians(geometry, image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=geometry,
        scale=10,
        maxPixels=1e13
    )

    # Extract values using getInfo()
    ndvi_median = stats.get('NDVI').getInfo()
    evi_median = stats.get('EVI').getInfo()
    ndwi_median = stats.get('NDWI').getInfo()

    return {
        'NDVI_median': ndvi_median,
        'EVI_median': evi_median,
        'NDWI_median': ndwi_median
    }

# Function to convert Shapely polygon to Earth Engine Geometry
def shapely_to_ee_geometry(shapely_geometry):
    if shapely_geometry.geom_type == 'Polygon' and shapely_geometry.exterior is not None:
        coords = list(shapely_geometry.exterior.coords)
        ee_geometry = ee.Geometry.Polygon(coords)
        return ee_geometry
    else:
        raise ValueError('Invalid Shapely geometry provided.')

# Loop through each Local Authority code
for la_code, shapefile_path in shapefile_paths.items():
    print(f'Processing Local Authority: {la_code}')

    # Load GeoDataFrame for the current Local Authority
    la_gdf = gpd.read_file(shapefile_path)

    # Load the corresponding image
    area_name = la_code_to_image[la_code]
    image = ee.Image(f'projects/ee-groundswelluk/assets/2024/{area_name}_Vegetation_Water_Indices')

    results_list = []
    # Iterate over each row in the GeoDataFrame
    for index, row in la_gdf.iterrows():
        shapely_geometry = row['geometry']

        # Check if the geometry is a valid Polygon
        if shapely_geometry.geom_type == 'Polygon' and shapely_geometry.exterior is not None:
            coords = list(shapely_geometry.exterior.coords)
            ee_geometry = ee.Geometry.Polygon(coords)

            #ee_geometry = shapely_to_ee_geometry(shapely_geometry)
            #medians = calculate_medians(ee_geometry, image)

            try:
                medians = calculate_medians(ee_geometry, image)

                result = {
                    'TOID': row['TOID'],
                    'lad23cd': row['lad23cd'],
                    'NDVI_median': medians['NDVI_median'],
                    'EVI_median': medians['EVI_median'],
                    'NDWI_median': medians['NDWI_median']
                }

                results_list.append(result)

                # Print progress for each buffer processed (optional)
                #print(f'Processed buffer {index + 1} of {len(la_gdf)}')

            except Exception as e:
                print(f'Error processing buffer {index + 1} for {la_code}: {str(e)}')

    # Save results for the current Local Authority as CSV
    result_df = pd.DataFrame(results_list)
    output_path = f'/content/drive/MyDrive/Papers/GroundsWell/WP4/Satellites/processed/{area_name}_Vegetation_Water_Indices_Medians.csv'
    result_df.to_csv(output_path, index=False)
    print(f'Saved results for Local Authority {la_code} to {output_path}')




Processing Local Authority: E06000049


Now that these are all finished, please run the R script `check_toids.R` which will assess whether there are any issues with the files and combine them into a single UPRN level indicator.

